In [29]:
import sqlite3
import pandas as pd
import json
import datetime
from tqdm import tqdm

# Connect to database
conn = sqlite3.connect('numero_data.sqlite')

# List all tables
df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)
print(df)


             name
0   film_metadata
1   indian_titles
2  sales_raw_data


In [30]:
tables = ['film_metadata', 'sales_raw_data', 'indian_titles']

In [31]:
# Columns
for table in tables:
    print(f"\n{table} columns:")
    df = pd.read_sql(f"PRAGMA table_info({table});", conn)
    display(df)
    print(f"\nTotal rows:")
    row_count = pd.read_sql(f"SELECT COUNT(*) as total_rows FROM {table};", conn)
    print(row_count['total_rows'][0])


film_metadata columns:


,cid,name,type,notnull,dflt_value,pk
0,0,numero_film_id,INTEGER,0,None,1
1,1,title,TEXT,0,None,0



Total rows:
100

sales_raw_data columns:


,cid,name,type,notnull,dflt_value,pk
0,0,numero_film_id,INTEGER,0,None,1
1,1,raw_json,TEXT,0,None,0



Total rows:
100

indian_titles columns:


,cid,name,type,notnull,dflt_value,pk
0,0,rank,INTEGER,0,None,0
1,1,title,TEXT,0,None,0
2,2,distributor,TEXT,0,None,0
3,3,release_date,TEXT,0,None,0
4,4,rating,TEXT,0,None,0
5,5,opening_screens,REAL,0,None,0
6,6,opening_screen_average,REAL,0,None,0
7,7,opening_weekend_total,REAL,0,None,0
8,8,opening_week_total,REAL,0,None,0
9,9,lifetime_total,REAL,0,None,0



Total rows:
214


In [32]:
# First 10 rows
for table in tables:
    print(f"\n{table}:")
    df_sample = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", conn)
    display(df_sample)


film_metadata:


,numero_film_id,title
0,48137,Noah's Ark
1,63815,Narivetta
2,64826,The Diplomat
3,66112,Bazooka
4,68343,Emergency



sales_raw_data:


,numero_film_id,raw_json
0,48137,"{""2025-02-20"": {""rows"": [{""boxOffice"": {""day1""..."
1,63815,"{""2025-05-22"": {""rows"": [{""boxOffice"": {""day1""..."
2,64826,"{""2025-03-13"": {""rows"": [{""boxOffice"": {""day1""..."
3,66112,"{""2025-04-10"": {""rows"": [{""boxOffice"": {""day1""..."
4,68343,"{""2025-01-16"": {""rows"": [{""boxOffice"": {""day1""..."



indian_titles:


,rank,title,distributor,release_date,rating,opening_screens,opening_screen_average,opening_weekend_total,opening_week_total,lifetime_total
0,1,Kantara A Legend: Chapter 1,Zstars Entertainment,2025-10-02,MA15+,100.0,9957.0,621261.59,995729.07,1821480.20
1,2,Saiyaara,Mindblowing,2025-07-18,M,59.0,6874.0,143460.28,405556.68,1763920.24
2,3,Chhaava,Mindblowing,2025-02-14,MA15+,93.0,7277.0,441056.77,676800.14,1675095.00
3,4,Coolie,Tolly Movies,2025-08-14,MA15+,141.0,9671.0,1271871.07,1363557.81,1446410.71
4,5,L2: Empuraan,Cyber Systems,2025-03-27,MA15+,142.0,9522.0,1233067.67,1352170.50,1404662.94


## Data Quality

In [33]:
# FUNCTIONS
# Get all column names for a table
def get_all_columns(conn, table_name):
    try:
        df_schema = pd.read_sql(f"PRAGMA table_info({table_name});", conn)
        return df_schema['name'].tolist()
    except Exception as e:
        print(f"Error getting columns for {table_name}: {e}")
        return []

# Missing value
def check_missing_values(conn, table_name):
    print(f"\nMissing values for: {table_name}")
    columns = get_all_columns(conn, table_name)
    if not columns:
        return

    results = []
    for col in columns:
        query = f'SELECT COUNT(*) as null_count FROM {table_name} WHERE "{col}" IS NULL;'
        null_count = pd.read_sql(query, conn)['null_count'][0]
        
        if null_count > 0:
            results.append({
                'column': col,
                'null_count': null_count
            })

    if not results:
        print("No NULL values found")
    else:
        print(pd.DataFrame(results))

# Duplicates
def check_duplicate(conn, table_name):
    print(f"\nDuplicates for: {table_name}")
    columns = get_all_columns(conn, table_name)
    if not columns:
        return
    
    col_string = ", ".join([f'"{col}"' for col in columns]) # ['title', 'rating'] -> "title", "rating"
    query = f"""
    SELECT
        COUNT(*) as duplicate_groups
    FROM (
        SELECT {col_string}, COUNT(*) as group_count
        FROM {table_name}
        GROUP BY {col_string}
        HAVING group_count > 1
    ) as sub;
    """
    duplicates = pd.read_sql(query, conn)['duplicate_groups'][0]
    if duplicates == 0:
        print("No fully duplicate rows found")
    else:
        print(f"Found {duplicates} groups of fully duplicated rows")

def check_partial_duplicates(conn, table_name, key_column):
    """Checks for duplicate values in a specific key column"""
    print(f"\nDuplicates on key '{key_column}' for: {table_name}")
    
    query = f"""
    SELECT
        "{key_column}",
        COUNT(*) as count
    FROM
        {table_name}
    GROUP BY
        "{key_column}"
    HAVING
        count > 1
    LIMIT 10;
    """
    df_dupes = pd.read_sql(query, conn)
    if df_dupes.empty:
        print(f"No duplicate values found in '{key_column}'")
    else:
        print(f"WARNING: Found duplicate values in '{key_column}':")
        print(df_dupes)

# Consistency
def check_inconsistent_text (conn, table_name, text_column):
    query = f"""
    SELECT {text_column}, COUNT (*) as count
    FROM {table_name}
    GROUP BY {text_column}
    ORDER BY count DESC
    LIMIT 50;
    """
    inconsistent = pd.read_sql(query, conn)
    display(inconsistent)

# Data Type
def check_date_formats(conn, table_name, date_column_as_text):
    print(f"\nChecking Date Formats in '{date_column_as_text}' for: {table_name}")
    
    query = f"""
    SELECT
        LENGTH("{date_column_as_text}") as len,
        SUBSTR("{date_column_as_text}", 3, 1) as pos3,
        SUBSTR("{date_column_as_text}", 4, 1) as pos4,
        SUBSTR("{date_column_as_text}", 5, 1) as pos5,
        COUNT(*) as count
    FROM
        {table_name}
    WHERE
        "{date_column_as_text}" IS NOT NULL
    GROUP BY
        len, pos3, pos4, pos5
    ORDER BY
        count DESC;
    """
    df_formats = pd.read_sql(query, conn)
    print(df_formats)


In [34]:
# Missing value
for table in tables:
    check_missing_values(conn, table)


Missing values for: film_metadata
No NULL values found

Missing values for: sales_raw_data
No NULL values found

Missing values for: indian_titles
                   column  null_count
0                  rating           2
1         opening_screens           4
2  opening_screen_average           4
3   opening_weekend_total           4
4      opening_week_total           4


In [35]:
query = """
SELECT *
FROM indian_titles
WHERE
    rating IS NULL
    OR opening_screens IS NULL
    OR opening_screen_average IS NULL
    OR opening_weekend_total IS NULL
    OR opening_week_total IS NULL;
"""
df_missing_rows = pd.read_sql(query, conn)
    
if df_missing_rows.empty:
    print("No rows found with missing values")
else:
    print(f"Found {len(df_missing_rows)} rows with missing data:")
    display(df_missing_rows)

Found 6 rows with missing data:


,rank,title,distributor,release_date,rating,opening_screens,opening_screen_average,opening_weekend_total,opening_week_total,lifetime_total
0,141,Ravanaprabhu,Cyber Systems,2025-10-10,M,NaN,NaN,NaN,NaN,6496.12
1,167,Chota Mumbai (2007) RE: 2025,Cyber Systems,2025-06-14,EX,NaN,NaN,NaN,NaN,2246.21
2,177,Detective Ujjwalan,Cyber Systems,2025-05-30,None,1.0,1597.0,1513.79,1596.60,1596.60
3,195,Padai Thalaivan,Zstars Entertainment,2025-05-30,EX,NaN,NaN,NaN,NaN,477.91
4,201,Nikita Roy,Zstars Entertainment,2025-07-18,None,1.0,342.0,270.93,341.92,341.92
5,214,Dinasari,Zstars Entertainment,2025-02-17,EX,NaN,NaN,NaN,NaN,10.00


In [36]:
# Duplicates
for table in tables:
    check_duplicate(conn, table)


Duplicates for: film_metadata
No fully duplicate rows found

Duplicates for: sales_raw_data
No fully duplicate rows found

Duplicates for: indian_titles
No fully duplicate rows found


In [37]:
check_partial_duplicates(conn, 'indian_titles', 'title')
check_partial_duplicates(conn, 'film_metadata', 'title')


Duplicates on key 'title' for: indian_titles
No duplicate values found in 'title'

Duplicates on key 'title' for: film_metadata
No duplicate values found in 'title'


In [38]:
# Consistency
check_table = 'indian_titles'
text_cols = ['distributor', 'rating']

for col in text_cols:
    check_inconsistent_text (conn, check_table, col)

,distributor,count
0,Tolly Movies,38
1,Forum Distribution,28
2,Cyber Systems,27
3,Mindblowing,26
4,Home Screen Entertainment,23
5,Zstars Entertainment,20
6,Zee Studios,14
7,Wanderlust Films,11
8,White Hill,6
9,Moviegoers Entertainment,6


,rating,count
0,M,70
1,MA15+,58
2,EX,48
3,PG,33
4,G,3
5,None,2


In [39]:
# Join Integrity
# ID match between film_metadata and numero_film_id
query = """
    SELECT COUNT(DISTINCT s.numero_film_id) as mismatch_count
    FROM sales_raw_data as s
    LEFT JOIN film_metadata as f 
        ON s.numero_film_id = f.numero_film_id
    WHERE f.numero_film_id IS NULL;
"""
mismatch_count = pd.read_sql(query, conn)['mismatch_count'][0]

if mismatch_count == 0:
    print("All 'numero_film_id' values in 'sales_raw_data' exist in 'film_metadata'")
else:
    print(f"Found {mismatch_count} unique 'numero_film_id' value(s) in 'sales_raw_data' that are NOT in 'film_metadata'")

All 'numero_film_id' values in 'sales_raw_data' exist in 'film_metadata'


In [40]:
# Title match between indian_titles and film_metadata
query = """
SELECT i.title
FROM indian_titles as i
LEFT JOIN film_metadata as f
    ON TRIM(LOWER(i.title)) = TRIM(LOWER(f.title))
WHERE 
    f.title IS NULL;
"""
df_unmatched = pd.read_sql(query, conn)
print(f"Found {len(df_unmatched)} unmatched titles:")
print(df_unmatched.to_string())

Found 115 unmatched titles:
                                                  title
0                                             Noahs Ark
1                                            Maranamass
2                                     Prince and Family
3                                  Mere Husband Ki Biwi
4                                             Robinhood
5                                                Mithde
6                                           Ground Zero
7                                              Loveyapa
8                                                  3BHK
9                                         Aabeer Gulaal
10                                            Maareesan
11                                              Khaleja
12                                               Maaman
13                          Gangland: The City of Crime
14                                               Mirage
15                                           Jombieland
16                  

In [41]:
query = """
SELECT f.title
FROM film_metadata as f
LEFT JOIN indian_titles as i
    ON TRIM(LOWER(i.title)) = TRIM(LOWER(f.title))
WHERE 
    i.title IS NULL;
"""
df_unmatched = pd.read_sql(query, conn)
print(f"Found {len(df_unmatched)} unmatched titles:")
print(df_unmatched.to_string())

Found 1 unmatched titles:
        title
0  Noah's Ark


# Data Understanding

In [42]:
# Date range
query_range = """
SELECT 
    MIN(release_date) as earliest_date, 
    MAX(release_date) as latest_date 
FROM indian_titles
WHERE release_date IS NOT NULL AND release_date != '';
"""
df_range = pd.read_sql(query_range, conn)
print("Time Range (min/max):")
print(df_range)

Time Range (min/max):
  earliest_date latest_date
0    2025-01-02  2025-10-31


In [43]:
# Films per month
query_month = """
SELECT 
    SUBSTR(release_date, 1, 7) as month, 
    COUNT(*) as film_count 
FROM indian_titles
WHERE release_date IS NOT NULL AND release_date != ''
GROUP BY month
ORDER BY month DESC;
"""
df_month = pd.read_sql(query_month, conn)
print(df_month)

     month  film_count
0  2025-10          30
1  2025-09          17
2  2025-08          21
3  2025-07          20
4  2025-06          16
5  2025-05          25
6  2025-04          18
7  2025-03          17
8  2025-02          29
9  2025-01          21


In [44]:
# Summary for numeric cols
numeric_cols = [
    "rank", "opening_screens", "opening_screen_average", 
    "opening_weekend_total", "opening_week_total", "lifetime_total"]

quoted_cols = ", ".join([f'"{col}"' for col in numeric_cols])
query = f"SELECT {quoted_cols} FROM indian_titles;"
df_numerics = pd.read_sql(query, conn)
print(df_numerics.describe().to_markdown(floatfmt=".2f"))

|       |   rank |   opening_screens |   opening_screen_average |   opening_weekend_total |   opening_week_total |   lifetime_total |
|:------|-------:|------------------:|-------------------------:|------------------------:|---------------------:|-----------------:|
| count | 214.00 |            210.00 |                   210.00 |                  210.00 |               210.00 |           214.00 |
| mean  | 107.50 |             30.38 |                  2094.25 |                80530.15 |            106372.78 |        149291.96 |
| std   |  61.92 |             31.84 |                  2325.37 |               169628.87 |            210655.29 |        314553.57 |
| min   |   1.00 |              1.00 |                    19.00 |                   37.00 |                37.00 |            10.00 |
| 25%   |  54.25 |              5.00 |                   516.75 |                 2048.60 |              2787.25 |          2884.67 |
| 50%   | 107.50 |             21.00 |                  1214.5

## sales_raw_data

In [45]:
# raw_json field
raw_json_string = pd.read_sql(
    "SELECT raw_json FROM sales_raw_data WHERE numero_film_id = 48137 LIMIT 1;", 
    conn
).iloc[0, 0]
json_data = json.loads(raw_json_string)
print(json.dumps(json_data, indent=2))

{
  "2025-02-20": {
    "rows": [
      {
        "boxOffice": {
          "day1": {
            "previousAdmissions": {},
            "today": 9701
          },
          "day2": {
            "previousAdmissions": {},
            "today": 9952,
            "yesterday": 9701
          },
          "day3": {
            "previousAdmissions": {},
            "today": 63552,
            "yesterday": 9952
          },
          "day4": {
            "previousAdmissions": {},
            "today": 48450,
            "yesterday": 63552
          },
          "day5": {
            "previousAdmissions": {},
            "today": 2200,
            "yesterday": 48450
          },
          "day6": {
            "previousAdmissions": {},
            "today": 6149,
            "yesterday": 2200
          },
          "day7": {
            "previousAdmissions": {},
            "today": 6101,
            "yesterday": 6149
          },
          "week": {
            "gross": 146105
          },
     

In [46]:
# FUNCTION
def load_raw_sales(conn):
    df_raw = pd.read_sql("SELECT numero_film_id, raw_json FROM sales_raw_data;", conn)
    print(f"Loaded {len(df_raw)} raw film records.")
    return df_raw

def flatten_sales_json(df_raw):
    processed_data = []
    
    for index, row in tqdm(df_raw.iterrows(), total=df_raw.shape[0]):
        film_id = row['numero_film_id']
        
        try:
            # Load the JSON string into a Python
            json_data = json.loads(row['raw_json'])
            
            # Loop 1 - Iterate through the week start dates
            for week_start_date, week_content in json_data.items():
                
                # Loop 2 - Iterate through the list of cinemas/regions
                for cinema_row in week_content.get('rows', []):
                    
                    # Variables
                    state = cinema_row.get('state')
                    state_id = cinema_row.get('stateId')
                    region = cinema_row.get('region')
                    region_id = cinema_row.get('regionId')
                    city = cinema_row.get('city')
                    city_id = cinema_row.get('cityId')
                    theatre_name = cinema_row.get('theatre')
                    theatre_id = cinema_row.get('theatreId')
                    circuit_name = cinema_row.get('circuit')
                    circuit_id = cinema_row.get('circuitId')
                    cinema_rank_for_week = cinema_row.get('rank')
                    
                    # Nested release data (week level)
                    release_data = cinema_row.get('release', {}) 
                    cumulative_gross = release_data.get('cumulativeBoxOffice')
                    cumulative_admissions = release_data.get('cumulativePaidAdmissions')
                    film_rank_at_cinema_for_week = release_data.get('thisWeekRank')
                    films_in_cinema_this_week = release_data.get('thisWeekFilmCount') # <-- NEW
                    
                    #week/weekend summary data
                    box_office = cinema_row.get('boxOffice', {})
                    week_summary = box_office.get('week', {})
                    weekend_summary = box_office.get('weekend', {})
                    
                    week_gross_at_cinema = week_summary.get('gross') # <-- NEW
                    weekend_gross_at_cinema = weekend_summary.get('gross') # <-- NEW
                    
                    
                    # Loop 3 - Iterate through the daily box office data
                    for day_key, sales_data in box_office.items():
                        
                        try:
                            # If the key isn't a day -> skip
                            if not day_key.startswith('day'):
                                continue # skips 'week', 'weekend'

                            day_num = int(day_key.replace('day', '')) - 1
                            
                            week_dt = datetime.datetime.strptime(week_start_date, '%Y-%m-%d').date()
                            
                            # Actual sales date base on week_start_date
                            actual_sales_date = week_dt + datetime.timedelta(days=day_num)
                        
                        except Exception as e:
                            print(f"Could not parse date for {film_id}, {week_start_date}, {day_key}: {e}")
                            actual_sales_date = None
                            continue

                        # Store new data
                        processed_data.append({
                            'numero_film_id': film_id,
                            'week_start_date': week_start_date,
                            'actual_sales_date': actual_sales_date,
                            'gross_today': sales_data.get('today'),
                            'gross_yesterday': sales_data.get('yesterday'),
                            'paid_admissions': sales_data.get('paidAdmissions'), 
                            'state': state,
                            'region': region,
                            'city': city,
                            'theatre_name': theatre_name,
                            'circuit_name': circuit_name,
                            'cinema_rank_for_week': cinema_rank_for_week,
                            'film_rank_at_cinema_for_week': film_rank_at_cinema_for_week,
                            'films_in_cinema_this_week': films_in_cinema_this_week, 
                            'week_gross_at_cinema': week_gross_at_cinema,
                            'weekend_gross_at_cinema': weekend_gross_at_cinema, 
                            'cumulative_gross_at_cinema': cumulative_gross,
                            'cumulative_admissions_at_cinema': cumulative_admissions,
                            'state_id': state_id,
                            'region_id': region_id,
                            'city_id': city_id,
                            'theatre_id': theatre_id,
                            'circuit_id': circuit_id,
                        })
                        
        except json.JSONDecodeError:
            print(f"Could not parse JSON for numero_film_id {film_id} -> Skipping")
        except Exception as e:
            print(f"An error occurred processing film {film_id}: {e}")

    print(f"\nSuccessfully processed {len(processed_data)} rows")
    
    # Convert into final DataFrame
    return pd.DataFrame(processed_data)

if __name__ == "__main__":
    df_raw = load_raw_sales(conn)
    if not df_raw.empty:
        df_clean = flatten_sales_json(df_raw)
        output_filename = 'sales_processed.csv'
        df_clean.to_csv(output_filename, index=False)
        print(df_clean.head())
        
    else:
        print("Error")

Loaded 100 raw film records.


100%|██████████| 100/100 [00:01<00:00, 69.52it/s]



Successfully processed 65947 rows
   numero_film_id week_start_date actual_sales_date  gross_today  \
0           48137      2025-02-20        2025-02-20       9701.0   
1           48137      2025-02-20        2025-02-21       9952.0   
2           48137      2025-02-20        2025-02-22      63552.0   
3           48137      2025-02-20        2025-02-23      48450.0   
4           48137      2025-02-20        2025-02-24       2200.0   

   gross_yesterday paid_admissions                      state  region  \
0              NaN            None  New South Wales (inc ACT)  Sydney   
1           9701.0            None  New South Wales (inc ACT)  Sydney   
2           9952.0            None  New South Wales (inc ACT)  Sydney   
3          63552.0            None  New South Wales (inc ACT)  Sydney   
4          48450.0            None  New South Wales (inc ACT)  Sydney   

                city  theatre_name  ... films_in_cinema_this_week  \
0  Parramatta & Ryde  Macquarie 16  ...         

In [47]:
sales = pd.read_csv('sales_processed.csv')
sales.head(5)

,numero_film_id,week_start_date,actual_sales_date,gross_today,gross_yesterday,paid_admissions,state,region,city,theatre_name,...,films_in_cinema_this_week,week_gross_at_cinema,weekend_gross_at_cinema,cumulative_gross_at_cinema,cumulative_admissions_at_cinema,state_id,region_id,city_id,theatre_id,circuit_id
0,48137,2025-02-20,2025-02-20,9701.0,NaN,NaN,New South Wales (inc ACT),Sydney,Parramatta & Ryde,Macquarie 16,...,24.0,146105.0,131655.0,146105,89.0,2,26,246,225,4
1,48137,2025-02-20,2025-02-21,9952.0,9701.0,NaN,New South Wales (inc ACT),Sydney,Parramatta & Ryde,Macquarie 16,...,24.0,146105.0,131655.0,146105,89.0,2,26,246,225,4
2,48137,2025-02-20,2025-02-22,63552.0,9952.0,NaN,New South Wales (inc ACT),Sydney,Parramatta & Ryde,Macquarie 16,...,24.0,146105.0,131655.0,146105,89.0,2,26,246,225,4
3,48137,2025-02-20,2025-02-23,48450.0,63552.0,NaN,New South Wales (inc ACT),Sydney,Parramatta & Ryde,Macquarie 16,...,24.0,146105.0,131655.0,146105,89.0,2,26,246,225,4
4,48137,2025-02-20,2025-02-24,2200.0,48450.0,NaN,New South Wales (inc ACT),Sydney,Parramatta & Ryde,Macquarie 16,...,24.0,146105.0,131655.0,146105,89.0,2,26,246,225,4


In [48]:
sales.shape

(65947, 23)

In [49]:
sales.dtypes

numero_film_id                       int64
week_start_date                     object
actual_sales_date                   object
gross_today                        float64
gross_yesterday                    float64
paid_admissions                    float64
state                               object
region                              object
city                                object
theatre_name                        object
circuit_name                        object
cinema_rank_for_week                 int64
film_rank_at_cinema_for_week       float64
films_in_cinema_this_week          float64
week_gross_at_cinema               float64
weekend_gross_at_cinema            float64
cumulative_gross_at_cinema           int64
cumulative_admissions_at_cinema    float64
state_id                             int64
region_id                            int64
city_id                              int64
theatre_id                           int64
circuit_id                           int64
dtype: obje

In [50]:
# Data type
sales['week_start_date'] = pd.to_datetime(sales['week_start_date'], errors='coerce')
sales['actual_sales_date'] = pd.to_datetime(sales['actual_sales_date'], errors='coerce')
categorical_cols = [
        'state', 
        'region', 
        'city', 
        'theatre_name', 
        'circuit_name'
    ]
for col in categorical_cols:
    sales[col] = sales[col].astype('category')
sales['film_rank_at_cinema_for_week'] = sales['film_rank_at_cinema_for_week'].astype('Int64')
sales['films_in_cinema_this_week'] = sales['films_in_cinema_this_week'].astype('Int64')
sales['cumulative_gross_at_cinema'] = sales['cumulative_gross_at_cinema'].astype('float64')

In [51]:
sales.dtypes

numero_film_id                              int64
week_start_date                    datetime64[ns]
actual_sales_date                  datetime64[ns]
gross_today                               float64
gross_yesterday                           float64
paid_admissions                           float64
state                                    category
region                                   category
city                                     category
theatre_name                             category
circuit_name                             category
cinema_rank_for_week                        int64
film_rank_at_cinema_for_week                Int64
films_in_cinema_this_week                   Int64
week_gross_at_cinema                      float64
weekend_gross_at_cinema                   float64
cumulative_gross_at_cinema                float64
cumulative_admissions_at_cinema           float64
state_id                                    int64
region_id                                   int64


In [52]:
# Missing value
cols_missing_values = sales.isna().sum()
cols_missing_values[cols_missing_values > 0]

gross_today                        18985
gross_yesterday                    22277
paid_admissions                    65947
film_rank_at_cinema_for_week        2737
films_in_cinema_this_week             42
week_gross_at_cinema                2737
weekend_gross_at_cinema             5957
cumulative_admissions_at_cinema      875
dtype: int64

In [53]:
metric_cols = sales.select_dtypes(include=['float64']).columns.tolist()
summary_table = sales[metric_cols].describe()
summary_table

,gross_today,gross_yesterday,paid_admissions,week_gross_at_cinema,weekend_gross_at_cinema,cumulative_gross_at_cinema,cumulative_admissions_at_cinema
count,4.696200e+04,4.367000e+04,0.0,6.321000e+04,5.999000e+04,6.594700e+04,65072.000000
mean,6.658301e+04,7.065901e+04,NaN,3.462759e+05,2.757480e+05,7.846130e+05,454.096923
std,1.374425e+05,1.417196e+05,NaN,6.345161e+05,5.164611e+05,1.429666e+06,781.737519
min,0.000000e+00,0.000000e+00,NaN,8.000000e+02,1.000000e+02,0.000000e+00,1.000000
25%,7.470250e+03,8.600000e+03,NaN,5.075100e+04,4.135700e+04,8.604400e+04,57.000000
50%,2.510000e+04,2.800000e+04,NaN,1.413375e+05,1.156340e+05,2.755320e+05,174.000000
75%,6.993725e+04,7.539775e+04,NaN,3.639370e+05,2.891040e+05,8.406980e+05,502.000000
max,4.022010e+06,4.022010e+06,NaN,1.172514e+07,1.058257e+07,1.607084e+07,8383.000000


In [ ]:
# Close connection
conn.close()